In [1]:
import tensorflow as tf
import numpy as np
import sys

from datetime import datetime

python_root = '../'
sys.path.insert(0, python_root)
from model.alexnet import AlexNet
from model.triplet_loss import batch_all_triplet_loss, batch_hardest_triplet_loss

In [2]:
np.random.seed(1)

## Model

In [3]:
## Configuration settings
import os
# Path to the textfiles for the trainings and validation set
train_file = '../train.txt'
val_file = '../val.txt'
test_file = '../test.txt'

image_type = 'bgr'
train_set_mean = np.array([22.47429166, 20.13914579, 5.62511388])
# train_set_mean = get_train_set_mean(train_file, image_type)
print("Mean of train set: {}".format(train_set_mean))

# Learning params
learning_rate = 0.01
num_epochs = 30
batch_size = 128
margin = 0.2
triplet_strategy = "batch_all"
# triplet_strategy = "batch_hard"

# Network params
dropout_rate = 0.5
no_train_layers = []

# How often we want to write the tf.summary data to disk
display_step = 1

# Path for tf.summary.FileWriter and to store model checkpoints
filewriter_path = "../logs/tmp/triplet_loss_{}".format(triplet_strategy)
checkpoint_path = "../logs/tmp/triplet_loss_{}_ckpt".format(triplet_strategy)

# Create parent path if it doesn't exist
if not os.path.isdir(checkpoint_path): os.makedirs(checkpoint_path)

Mean of train set: [22.47429166 20.13914579  5.62511388]


In [4]:
# TF placeholder for graph input and output
x = tf.placeholder(tf.float32, [batch_size, 227, 227, 3])
labels = tf.placeholder(tf.float32, [batch_size, 4])
keep_prob = tf.placeholder(tf.float32)

# Initialize model
skip_layers = ['fc8']  # Don't use weights from AlexNet
model = AlexNet(x, keep_prob, skip_layers, weights_path='../model/bvlc_alexnet.npy')

# Link variable to model output
embeddings = tf.nn.l2_normalize(model.fc8, axis=1)

# List of trainable variables of the layers we want to train
var_list = [v for v in tf.trainable_variables() if v.name.split('/')[0] not in no_train_layers]

with tf.name_scope("triplet_loss"):
    if triplet_strategy == "batch_all":
        loss, fraction = batch_all_triplet_loss(labels, embeddings, margin=margin, squared=False)
    elif triplet_strategy == "batch_hard":
        loss = batch_hardest_triplet_loss(labels, embeddings, margin=margin, squared=False)
    else:
        raise ValueError("Triplet strategy not recognized: {}".format(triplet_strategy))

In [5]:
# Train op
with tf.name_scope("train"):
  # Get gradients of all trainable variables
    gradients = tf.gradients(loss, var_list)
    gradients = list(zip(gradients, var_list))

    # Create optimizer and apply gradient descent to the trainable variables
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.apply_gradients(grads_and_vars=gradients)

In [6]:
# Add gradients to summary
for gradient, var in gradients:
    tf.summary.histogram(var.name + '/gradient', gradient)

# Add the variables we train to the summary
for var in var_list:
    tf.summary.histogram(var.name, var)

# Add the loss to summary
if triplet_strategy == "batch_all":
    tf.summary.scalar('triplet_loss', loss)
    tf.summary.scalar('fraction_positive_triplets', fraction)
elif triplet_strategy == "batch_hard":
    tf.summary.scalar('triplet_loss', loss)

# Add embedding_mean_norm to summary
embedding_mean_norm = tf.reduce_mean(tf.norm(embeddings, axis=1))
tf.summary.scalar("embedding_mean_norm", embedding_mean_norm)

merged_summary = tf.summary.merge_all()

# Initialize the FileWriter
writer = tf.summary.FileWriter(filewriter_path)

# Initialize an saver for store model checkpoints
saver = tf.train.Saver()

INFO:tensorflow:Summary name conv1/weights:0/gradient is illegal; using conv1/weights_0/gradient instead.
INFO:tensorflow:Summary name conv1/biases:0/gradient is illegal; using conv1/biases_0/gradient instead.
INFO:tensorflow:Summary name conv2/weights:0/gradient is illegal; using conv2/weights_0/gradient instead.
INFO:tensorflow:Summary name conv2/biases:0/gradient is illegal; using conv2/biases_0/gradient instead.
INFO:tensorflow:Summary name conv3/weights:0/gradient is illegal; using conv3/weights_0/gradient instead.
INFO:tensorflow:Summary name conv3/biases:0/gradient is illegal; using conv3/biases_0/gradient instead.
INFO:tensorflow:Summary name conv4/weights:0/gradient is illegal; using conv4/weights_0/gradient instead.
INFO:tensorflow:Summary name conv4/biases:0/gradient is illegal; using conv4/biases_0/gradient instead.
INFO:tensorflow:Summary name conv5/weights:0/gradient is illegal; using conv5/weights_0/gradient instead.
INFO:tensorflow:Summary name conv5/biases:0/gradient i

In [7]:
from model.datagenerator import ImageDataGenerator
train_generator = ImageDataGenerator(train_file,
                                     horizontal_flip = False, shuffle = True)
val_generator = ImageDataGenerator(val_file, shuffle = False)
test_generator = ImageDataGenerator(test_file, shuffle = False, mean=np.array([22.47429166, 20.13914579, 5.62511388]))

In [8]:
# Get the number of training/validation steps per epoch
train_batches_per_epoch = np.floor(train_generator.data_size / batch_size).astype(np.int16)
val_batches_per_epoch = np.floor(val_generator.data_size / batch_size).astype(np.int16)

# Start training

You can also skip training and use the checkpoint at 30 epochs in `../logs/model_epoch30.ckpt`. See [Embeddings](#Embeddings).

In [9]:
with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
    
    # Load the pretrained weights into the non-trainable layer
    model.load_initial_weights(sess)
    
    print("{} Start training...".format(datetime.now()))
    print("{} Open Tensorboard at --logdir {}".format(datetime.now(), filewriter_path))
    
    # Loop over number of epochs
    for epoch in range(num_epochs):

        print("{} Epoch number: {}".format(datetime.now(), epoch+1))

        step = 1

        while step < train_batches_per_epoch:

            # Get a batch of images and labels
            batch_x_train, batch_labels_train = train_generator.next_batch(batch_size)

            # And run the training op
            sess.run(train_op, feed_dict={x: batch_x_train,
                                          labels: batch_labels_train,
                                          keep_prob: dropout_rate})

            # Generate summary with the current batch of data and write to file
            if step%display_step == 0:
                s = sess.run(merged_summary, feed_dict={x: batch_x_train,
                                                        labels: batch_labels_train,
                                                        keep_prob: 1.})
                writer.add_summary(s, epoch*train_batches_per_epoch + step)

            step += 1

        # Validate the model on the entire validation set
        print("{} Start validation".format(datetime.now()))
        loss_val = 0.
        test_count = 0
        for _ in range(val_batches_per_epoch):
            batch_tx, batch_ty = val_generator.next_batch(batch_size)
            loss_current = sess.run(loss, feed_dict={x: batch_tx,
                                                     labels: batch_ty,
                                                     keep_prob: 1.})
            loss_val += loss_current
            test_count += 1
        loss_val = loss_val / test_count
        print("{} Average loss for validation set = {:.4f}".format(datetime.now(), loss_val))
        
        # Reset the file pointer of the image data generator
        val_generator.reset_pointer()
        train_generator.reset_pointer()
        
        print("{} Saving checkpoint of model...".format(datetime.now()))
        #save checkpoint of the model
        checkpoint_name = os.path.join(checkpoint_path, image_type+'_model_epoch'+str(epoch+1)+'.ckpt')
        save_path = saver.save(sess, checkpoint_name)

        print("{} Model checkpoint saved at {}".format(datetime.now(), checkpoint_name))


2018-10-16 10:11:40.449031 Start training...
2018-10-16 10:11:40.449309 Open Tensorboard at --logdir ../logs/tmp/triplet_loss_batch_all
2018-10-16 10:11:40.449436 Epoch number: 1
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_114/rgb/110.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_84/rgb/50.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_77/rgb/26.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_42/rgb/74.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_272/rgb/59.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_229/rgb/67.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_235/rgb/131.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_too

Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_157/rgb/71.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_207/rgb/108.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_20/rgb/21.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_102/rgb/30.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_295/rgb/31.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_164/rgb/11.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_285/rgb/59.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_284/rgb/72.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_147/rgb/31.png
Here img_path is :
/home/fra

Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_60/rgb/29.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_177/rgb/51.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_145/rgb/6.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_275/rgb/15.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_22/rgb/44.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_230/rgb/113.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_237/rgb/28.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_59/rgb/6.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_29/rgb/44.png
Here img_path is :
/home/francesc

Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_97/rgb/86.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_17/rgb/81.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_35/rgb/16.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_114/rgb/62.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_232/rgb/53.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_252/rgb/35.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_89/rgb/12.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_27/rgb/55.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_150/rgb/80.png
Here img_path is :
/home/francesc

Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_90/rgb/43.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_235/rgb/125.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_104/rgb/76.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_160/rgb/45.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_202/rgb/26.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_149/rgb/38.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_90/rgb/72.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_65/rgb/19.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_129/rgb/11.png
Here img_path is :
/home/franc

...

Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_17/rgb/75.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_239/rgb/2.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_272/rgb/72.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_214/rgb/40.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_264/rgb/51.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_229/rgb/2.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_217/rgb/5.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_270/rgb/30.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_84/rgb/19.png
Here img_path is :
/home/francesc

Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_195/rgb/1.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_2/rgb/15.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_57/rgb/16.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_50/rgb/46.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_152/rgb/89.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_164/rgb/70.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_140/rgb/20.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_262/rgb/62.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_117/rgb/46.png
Here img_path is :
/home/francesc

Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_40/rgb/0.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_265/rgb/23.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_37/rgb/10.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_172/rgb/33.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_279/rgb/3.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_149/rgb/59.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_0/rgb/102.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_197/rgb/12.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_27/rgb/52.png
Here img_path is :
/home/francesco

Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_247/rgb/3.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_49/rgb/37.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_27/rgb/35.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_287/rgb/81.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_234/rgb/1.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_95/rgb/102.png
Here img_path is :
/home/francesco/catkin_extended_ws/src/line_tools/data/train/traj_1/frame_219/rgb/70.png


KeyboardInterrupt: 